# SDK Examples - Querying Data Models

Some of the most common query operators in the Sight Machine SDK. We'll use the demo environment (https://demo.sightmachine.io).

In [35]:
from smsdk import client
from datetime import datetime, timedelta
import pandas as pd

## Initialize the SDK client and get a list of all machine types

In [36]:
api_key = ''
api_secret = ''
cli = client.Client('demo')
cli.login('apikey', 
          key_id = api_key, 
          secret_id = api_secret)

types = cli.get_machine_type_names()
types

['Diecast', 'Fusion', 'Lasercut', 'Pick & Place']

# Working with Cycles

Cycles are the core data set in the SM Platform.  Cycles represent a unit of work on a machine and will contain a variety of data from sensors, quality managent systems, ERP, MES, etc.  

Each cycle is associate with a Machine and a range of time.  Each Machine has a machine type which determines the data schema.  So to query for cycle data, the first step is to lookup the machine type and then to lookup the specific machine(s) of that type.

In [37]:
# list machines of a specific type
machine_type = types[2]
machines = cli.get_machine_names(source_type=machine_type)
machines

['Abidjan - Lasercut 1',
 'Abidjan - Lasercut 2',
 'Abidjan - Lasercut 3',
 'Bantam City - Lasercut 1',
 'Bantam City - Lasercut 2',
 'Bantam City - Lasercut 3',
 'Carmel - Lasercut 1',
 'Carmel - Lasercut 2',
 'Carmel - Lasercut 3',
 'Carmel - Lasercut 4',
 'Carmel - Lasercut 5',
 'Carmel - Lasercut 6',
 'Lima - Lasercut 1',
 'Lima - Lasercut 2',
 'Santa Catarina - Lasercut 1',
 'Santa Catarina - Lasercut 2',
 'Santa Catarina - Lasercut 3',
 'Singapore - Lasercut 1',
 'Singapore - Lasercut 2',
 'Singapore - Lasercut 3',
 'Singapore - Lasercut 4']

In [38]:
# retrieve the schema for a particular machine (more on this at end of notebook)
# extract only a list of tag display names
columns = cli.get_machine_schema(machines[0])['display'].to_list()
columns

['Machine',
 'Cycle Start Time',
 'Cycle End Time',
 'Production Day',
 'Cycle Time (Net)',
 'Cycle Time (Gross)',
 'Shift',
 'Output',
 'Alarms',
 'BLOCKED',
 'Conveyor Input - Total Time',
 'Conveyor Output - Total Time',
 'Conveyor Speed',
 'DOWN',
 'Defect Category',
 'Defect Reason',
 'Downtime Category',
 'Downtime Reason',
 'Gripper Make',
 'Gripper Model',
 'Laser Current',
 'Laser Cutting - Total Time',
 'Laser Voltage',
 'Operator Load - Total Time',
 'Product SKU',
 'Robot Make',
 'Robot Model',
 'Robot Velocity (x)',
 'Robot Velocity (y)',
 'Robot Velocity (z)',
 'Operator Unload - Total Time',
 'Vision System - Total Time',
 'Downtime Type',
 'Scrap Quantity',
 'Serial']

### Selecting a Particular Development Pipeline schema

You can select a development pipeline schema using following code example. This works very similarly to the 'in-use' feature in MA - we can select an alternate pipeline to treat as the production one. Similarly to MA, the setting will persist until you change it back or create a new client.

*Note: By default, the production pipeline schema will be used (just like in MA).*

In [39]:
db_schema = 'pipeline_id' 
cli.select_db_schema(schema_name=db_schema)

## A basic starting query.

Once you have a machine type and machine, you can start to query for cycle data.  We'll use variations on this theme to demonstrate different query options and their effects.

Note that this baseline query already demonstrates:
- Basic filter rules formatted as key value pairs
- Filtering for greater than or less than values
    - It uses `__gte` for greater than or equal.  Use `__gt` for greater than.  Similarly `__lte` is less than or equal vs. `__lt` for less than.
- Sorting returned results
    - Note the `-` prefix before `Endtime` means to sort descending.  To sort ascending, do not place a prefix in front of the variable name.

In [40]:
query = {'Machine': machines[5],
         'End Time__gte' : datetime(2023, 4, 1), 
         'End Time__lte' : datetime(2023, 4, 2), 
         '_order_by': '-End Time'}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
df.head()

_only not specified.  Selecting first 50 fields.
Size of returned data: (1383, 35)


,Machine,Start Time,End Time,Production Day,Cycle Time (Net),Cycle Time (Gross),Shift,Output,Alarms,BLOCKED,...,Robot Make,Robot Model,Robot Velocity (x),Robot Velocity (y),Robot Velocity (z),Operator Unload - Total Time,Vision System - Total Time,Downtime Type,Scrap Quantity,Serial
_id,,,,,,,,,,,,,,,,,,,,,
CgXYWBssr6gDrUvBC6HPRK8I41DAblqWvXcPDIugZ9A=,Bantam City - Lasercut 3,2023-04-01 23:58:36.008,2023-04-01 23:59:24.008,2023-04-01,48000.0,48000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,0.112596,0.072146,0.129064,1000.0,1000.0,None,0.0,JB_BT_Lasercut_3/aMpVC8AUSV6Z4gglHjIUiw.1086
zmr6F1MAnJnrA+d73ZqZBTf8RyTxpd4cETwHf/v2Zos=,Bantam City - Lasercut 3,2023-04-01 23:57:46.008,2023-04-01 23:58:35.008,2023-04-01,49000.0,49000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,-0.316847,0.037143,0.198495,1000.0,1000.0,None,0.0,JB_BT_Lasercut_3/aMpVC8AUSV6Z4gglHjIUiw.1085
1ViZHLv4j3DRp2nJy3wwJ5aeW0LGS8cgPeDDgqbvZss=,Bantam City - Lasercut 3,2023-04-01 23:56:57.008,2023-04-01 23:57:45.008,2023-04-01,48000.0,48000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,0.319940,0.050132,0.134968,1000.0,1000.0,None,0.0,JB_BT_Lasercut_3/aMpVC8AUSV6Z4gglHjIUiw.1084
7NeUkBFvemGa8LStPysJMSDzPpYE7AGqrAZp5dSPBTA=,Bantam City - Lasercut 3,2023-04-01 23:56:07.008,2023-04-01 23:56:56.008,2023-04-01,49000.0,49000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,0.442771,-0.354746,0.086758,1000.0,1000.0,None,0.0,JB_BT_Lasercut_3/aMpVC8AUSV6Z4gglHjIUiw.1083
9LPVx1u8pkLjxS71Lav7Xo0Yl+TkaGx6/PUKAFOOCtk=,Bantam City - Lasercut 3,2023-04-01 23:55:19.008,2023-04-01 23:56:06.008,2023-04-01,47000.0,47000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,0.302452,0.148058,0.039285,1000.0,1000.0,None,0.0,JB_BT_Lasercut_3/aMpVC8AUSV6Z4gglHjIUiw.1082


# Selecting columns and silencing the `_only` Warning

To select a specific set of columns, provide a list of column names as a value for the key _only.  For example, `'_only': ['column1', 'column2', 'column3']`

If you do not use _only, the SDK will automatically select the first 50 stats in the machine's configuration, plus common metadata fields for the query.  

Note, you can also pass `'_only': '*'`, which will return everything, including a large number of internal fields.  Since this includes may fields you probably will not need, expect the resulting queries to be quite slow.

**IMPORTANT** If a selected column is all null, it will not be included in the returned data frame.  If you are getting fewer columns returned than expected, this mostly likely means that there was only null data for that column.



In [41]:
# Get the first 10 columns, plus Machine and End Time
select_columns = ['Machine', 'End Time'] + columns[:5]

query = {'Machine': machines[0],
         'End Time__gte' : datetime(2023, 4, 1), 
         'End Time__lte' : datetime(2023, 4, 2),  
         '_order_by': '-End Time',
         '_only': select_columns}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
df.head()

None
Size of returned data: (0, 0)


Traceback (most recent call last):
  File "/Users/akp/opt/anaconda3/envs/sdk-only/lib/python3.11/site-packages/smsdk/ma_session.py", line 180, in _get_records_v1
    raise ValueError("Error - {}".format(response.text))
ValueError: Error - {"description":"An unexpected error occurred.","details":{},"error":"server_error"}



""


## Restricting the number of rows returned with `_limit` and `_offset`

To restrict the number of rows, use the _limit query option.  For example, `'_limit': 500`.  This will then return at most 500 rows.  

To skip over a specified number of rows, use the _offset query option.  For example `'_offset': 50`.

It is fairly common to use a combination of _limit and _offset togheter for applications such as paginating data.  For example, if a query would normally return 100 rows and you want to break it into two queries you could return the first 50 rows with `'_offset': 0, '_limit': 50` and then return the second 50 rows with `'_offset': 50, '_limit': 50`.

In [42]:
query = {'Machine': machines[0],
         'End Time__gte' : datetime(2023, 4, 1), 
         'End Time__lte' : datetime(2023, 4, 2),  
         '_order_by': '-End Time',
         '_offset': 10,
         '_limit': 500}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')

# Notice in the returned data set that the first row is at 23:50 instead of midnight, becuase of the offset
df.head()

_only not specified.  Selecting first 50 fields.
Size of returned data: (500, 35)


,Machine,Start Time,End Time,Production Day,Cycle Time (Net),Cycle Time (Gross),Shift,Output,Alarms,BLOCKED,...,Robot Make,Robot Model,Robot Velocity (x),Robot Velocity (y),Robot Velocity (z),Operator Unload - Total Time,Vision System - Total Time,Downtime Type,Scrap Quantity,Serial
_id,,,,,,,,,,,,,,,,,,,,,
AxSB99asDu8Qhh7QK5g8K9eghBAW8MvcvXOVUJPxizs=,Abidjan - Lasercut 1,2023-04-01 23:46:06.013,2023-04-01 23:47:23.013,2023-04-01,77000.0,77000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,0.382599,0.096932,0.114036,1000.0,1000.0,None,0.0,JB_AB_Lasercut_1/Krl9cwxTTG2F3OqMsiFzOQ.1022
Cusge3/JBVVvA1opc6IcLIiZyG2MeA87rMJmyKCvRus=,Abidjan - Lasercut 1,2023-04-01 23:44:48.013,2023-04-01 23:46:05.013,2023-04-01,77000.0,77000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,0.495162,-0.094539,0.140866,1000.0,1000.0,None,0.0,JB_AB_Lasercut_1/Krl9cwxTTG2F3OqMsiFzOQ.1021
NDXGWvYOj9R1nNXaT3SMuagP4708eENscS9IY8Rk4j4=,Abidjan - Lasercut 1,2023-04-01 23:43:30.013,2023-04-01 23:44:47.013,2023-04-01,77000.0,77000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,-0.111663,-0.232308,-0.716481,1000.0,1000.0,None,0.0,JB_AB_Lasercut_1/Krl9cwxTTG2F3OqMsiFzOQ.1020
V/TaFJq1BMFibfb1RLbqmD47/FOAX61STWeAZZFLcTo=,Abidjan - Lasercut 1,2023-04-01 23:42:12.013,2023-04-01 23:43:29.013,2023-04-01,77000.0,77000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,0.007134,-0.096622,0.146825,1000.0,1000.0,None,0.0,JB_AB_Lasercut_1/Krl9cwxTTG2F3OqMsiFzOQ.1019
/o9queOZWffdawIJBOC8jTnorJgeonm87aO2ke27S2g=,Abidjan - Lasercut 1,2023-04-01 23:40:52.013,2023-04-01 23:42:11.013,2023-04-01,79000.0,79000.0,Shift C,1.0,None,0.0,...,FANUC,LR MATE 200,-0.120086,0.233366,-0.094002,1000.0,1000.0,None,0.0,JB_AB_Lasercut_1/Krl9cwxTTG2F3OqMsiFzOQ.1018


# Data from more than one Machine or filtering by a list of values using `__in`

Filters can specify a list of acceptable values.  This is most commonly used when selecting data from more than one machine, though it can be used on any field name.  This is done by appending `__in` (*note two underscores*) to the column name and then specifying the list of options.  For example:

    'Machine__in': ['Oven1', 'Oven2']

or

    'Status__in': ['Idle', 'Maintenance', 'Down']

**Important** Selecting multiple machines of different types can result in spare and confusing data frames.  It is strongly recommended to only pick multiple machines of the same type.

You can also query for values that are not in a list by using `__nin` with the same format as `__in`.  For example:

    'Product_Code__nin': ['SuperMax 5000', 'MegaValue 6000']

In [43]:
# # Note: taking the first three machines' data, so will result in three times as many records returned
# query = {'Machine__in': machines[0:3],
#          'End Time__gte' : datetime(2023, 4, 1), 
#          'End Time__lte' : datetime(2023, 4, 2),  
#          '_order_by': '-End Time'}
# df = cli.get_cycles(**query)

# print(f'Size of returned data: {df.shape}')
# # Notice the Machine column now has three different values
# df.head()

# NOTE currently doesn't work

# Filtering to only rows where a specified field exists with `__exists`

Some data fields, such as inspection data, are often quite sparse.  To filter to only rows with or without non-null values, use `__exists`.  `__exists` should be appended to the name of the field, and then give it a boolean for if you want the field to exist (True) or not exist (False).  For example:

    'Inspection_Value__exists': True

or

    'Failure_Code__exists': False

In [44]:
query = { 'Machine': machines[0],
         'DOWN__exists': True,
         'End Time__gte' : datetime(2023, 4, 1), 
         'End Time__lte' : datetime(2023, 4, 2), 
         '_order_by': '-End Time',
         '_only': ['Machine', 'End Time', 'DOWN'],
         '_limit': 100}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
# Query was for two machines, but BM: Axial Load not on the second, so that doesn't appear in the data set
df.head()

Size of returned data: (100, 3)


,Machine,End Time,DOWN
_id,,,
hiLGtZpGxlt3xqCT7WEFF9GBg7pymVEtASntQuN3KJ0=,Abidjan - Lasercut 1,2023-04-01 23:58:47.013,0.0
XGnwss0Aj7RNokYh3V0rK7NvJQK9Wb7oWN8il/rIVf8=,Abidjan - Lasercut 1,2023-04-01 23:57:29.013,0.0
tKZUIyqN1ZdUDNCvfS+8lpzike04V9yjI5SkcfFX1Eg=,Abidjan - Lasercut 1,2023-04-01 23:56:11.013,0.0
z6khQxXHZgq/XjXXR5KP9MGxdVUvTZQTMxj3+2GCEL0=,Abidjan - Lasercut 1,2023-04-01 23:54:53.013,0.0
B8qMqdkWGCuK7V71sE5gysCHtaeCku1SxW7qIr3qDMo=,Abidjan - Lasercut 1,2023-04-01 23:53:38.013,0.0


# Testing for inequality with `__ne`

The standard `key: value` format assumes it is testing when the key equals the value.  To change this to inequality, add a `__ne` suffix.  For example, `'StatusCode__ne': 0`



In [45]:
query = {'Machine': machines[0],
         'DOWN__ne': 0,
         'End Time__gte' : datetime(2023, 4, 1), 
         'End Time__lte' : datetime(2023, 4, 2), 
         '_order_by': '-End Time',
         '_only': ['Machine', 'End Time', 'DOWN']}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
df.head()

Size of returned data: (18, 3)


,Machine,End Time,DOWN
_id,,,
mbSFSa1oxxqJEZSOxF7EoAHCk1VqRyrx8Q0J9sDIRWA=,Abidjan - Lasercut 1,2023-04-01 20:02:52.013,1.0
R6nhF1tPKHvBLy9MZLveFepjI9104YvALHLt9LP5eCg=,Abidjan - Lasercut 1,2023-04-01 18:45:16.013,1.0
EsY+WGK76j9aDDHdYIu6At28WANGtliB7ZRWjK+TvrI=,Abidjan - Lasercut 1,2023-04-01 18:35:11.013,1.0
mwn38oDw4QCYUzKXpJpxMsMRtCtowF/uMiMNWZB9Yrg=,Abidjan - Lasercut 1,2023-04-01 18:08:09.013,1.0
1W7bvSpFXUlVTJ5r0pLTErwkWOKxyTgI3sIJwhJ1E+Q=,Abidjan - Lasercut 1,2023-04-01 17:39:47.013,1.0


# Working with Downtimes
Similarly to Cycles, the Downtime data model can be queried for a given machine. Everything from the above section still applies, but the main function is get_downtimes() as opposed to get_cycles.

In [46]:
query = {'Machine': machines[0],
         'End Time__gte' : datetime(2023, 4, 1), 
         'End Time__lte' : datetime(2023, 4, 2), 
         '_order_by': '-End Time'}
df = cli.get_downtimes(**query)

print(f'Size of returned data: {df.shape}')
df.head()

Size of returned data: (18, 8)


,Machine,Start Time,End Time,Duration,Shift,Downtime Reason,Downtime Category,Downtime Type
_id,,,,,,,,
TME9XzjnCx2aiGL+47++K+GqEoyAN6cVauD73chGog4=,Abidjan - Lasercut 1,2023-04-01 19:54:02.013,2023-04-01 20:02:52.013,530000.0,Shift C,Lost connectivity,Controller,unplanned
u7K7aH2urCwDUpu5hAaqoBgkwt/igQis5aMnt6RRtmk=,Abidjan - Lasercut 1,2023-04-01 18:36:26.013,2023-04-01 18:45:16.013,530000.0,Shift C,Head failure,Cutting Head,unplanned
jAO+ZSAluYoAf/ZoTtdO9KzB0CwBR+16aPpT+yRhgCk=,Abidjan - Lasercut 1,2023-04-01 18:26:21.013,2023-04-01 18:35:11.013,530000.0,Shift C,Lost connectivity,Controller,unplanned
19lGRAo3lbsQb9ugZQ+sOzsgVR9AdG84VpJZS1uuLGI=,Abidjan - Lasercut 1,2023-04-01 17:59:19.013,2023-04-01 18:08:09.013,530000.0,Shift C,Software fault,Controller,unplanned
1cn6WOq+JdyjFg1dLzcmtE0Z5mzEHxo5463Nd7NkphE=,Abidjan - Lasercut 1,2023-04-01 17:30:57.013,2023-04-01 17:39:47.013,530000.0,Shift C,Software fault,Controller,unplanned


# Working with Parts

Whereas Cycles contain data happening on a particular machine, Parts track an object across multiple machines.  The general structure for query parts is similar for working with cycles, though slightly simpler.  With a Cycle, the pattern is to find the Machine Type, then the Machine, then get Cycle data associated with the machine.  With Parts, you only need a two step process to look up Part Types and then Part data.


In [47]:
part_types = cli.get_part_type_names()
part_type = part_types[0]
part_types

['Fitting', 'Engine Block']

In [48]:
# look at parts schema, same as we did above for machine schema
columns = cli.get_part_schema(part_type)['display'].to_list()
columns[:10]

['Conveyor Input - Total Time',
 'Conveyor Output - Total Time',
 'Current',
 'DefectCategory',
 'DefectReason',
 'Fusion Process',
 'Gripper Make',
 'Gripper Model',
 'Gripper Rotation',
 'Humidity']

The options for querying parts are similar to querying for cycles - use the same operators described above.

In [49]:
query = {'Part': part_type,
         'End Time__gte' : datetime(2023, 4, 1), 
         'End Time__lte' : datetime(2023, 4, 2),
         'DefectReason__exists': True,
         '_limit': 10,
         '_only': columns[:30]}

df = cli.get_parts(**query)

print(f'Size of returned data: {df.shape}')
df.head()

Size of returned data: (10, 31)


,Part,Conveyor Input - Total Time,Conveyor Output - Total Time,Current,DefectCategory,DefectReason,Fusion Process,Gripper Make,Gripper Model,Gripper Rotation,...,Robot Velocity (z),Temperature,Operator Unload - Total Time,Voltage,factory,production_date,Conveyor Speed,Laser Current,Laser Cutting - Total Time,Laser Voltage
_id,,,,,,,,,,,,,,,,,,,,,
Qx22rC5JRo2OdnK4OBwkRHAUe46zK074hk6tze9lTOE=,Fitting,1000.0,1000.0,8.748493,Operator Stop,Misalignment,1000.0,SCHUNK,GSM-P 32-E-090,52.997685,...,21.562725,608.276616,1000.0,216.383445,HB,2023-04-02,NaN,NaN,NaN,NaN
G2WDqIIAt2YwnENrf2lE3prYLsoINNEAWabcnTV/O4U=,Fitting,1000.0,1000.0,NaN,Dimensions / Position,Incomplete cut,NaN,DESTACO,DPE-400-10,NaN,...,-0.018825,NaN,0.0,NaN,CA,2023-04-01,10.606061,13.423906,14000.0,8.955747
G0pb4tHcz2nnnC+7KlzrUlm7ylLtZPhBdwk8okXw51Q=,Fitting,1000.0,1000.0,NaN,Dimensions / Position,Off center,NaN,DESTACO,DPE-400-10,NaN,...,-0.215522,NaN,0.0,NaN,BT,2023-04-01,20.000000,21.971163,14000.0,14.626384
2RXesQbGE4In2mA1NUrkjxrU5SSlZMA/ehgbqgrqsbg=,Fitting,1000.0,1000.0,NaN,Dimensions / Position,Off center,NaN,DESTACO,DPE-400-10,NaN,...,0.758246,NaN,0.0,NaN,CA,2023-04-01,18.421053,23.092598,545000.0,15.403619
yK0teK04vCakU954pr3FX3QK1+pu3Zu/JefZ1gjqTjs=,Fitting,1000.0,1000.0,8.409947,Operator Stop,Misalignment,1000.0,SCHUNK,GSM-P 32-E-090,44.051869,...,20.051109,600.041671,1000.0,223.638173,DC,2023-04-01,NaN,NaN,NaN,NaN


# Machines and Machine Types
There is additional information about machines and machine types that can be queried from the SDK. This info can help you format or transform your queries to fit your needs. Examples are included below.

## Machine-Level Info

### Timezones

By default, all timestamps are in UTC.  To find the local timezone associated with a machine, use the ```get_machine_timezone``` function and provide the machine name.  This will then return the name of the timezone, which can be used with libraries such as pytz to convert time zones.

In [50]:
print(machines[0])
tz = cli.get_machine_timezone(machines[0])
print(tz)

Abidjan - Lasercut 1
Africa/Abidjan


### Get Machine Type from Machine
All machines can be grouped into machine types. You may need to programmatically look up the type of a machine given its name. To get the machine type using machine name (or display name), use ```cli.get_type_from_machine(machine_name)```.

In [51]:
cli.get_type_from_machine(machines[0])

'Lasercut'

### Get Machine Data Schema
The machine schema is a table containing metadata about the tags included in cycle data for a particular machine. It can be retrieved with ```cli.get_machine_schema(machine_name)```. There are a few additional optional parameters that can be passed to the function:
- ```types```: list of strings specifying a subset of column data types that you want to see.
    - ```cli.get_machine_schema(machine_name, types=['continuous'])```
- ```show_hidden```: (default = False) set to True to see the few additional fields that are hidden by default both here and in MA.

In [59]:
# retrieve the schema for a particular machine
schema = cli.get_machine_schema(machines[0])
schema.head()

,display,unit,sight_type,type,stream_types,raw_data_field,name,formatting,annotations,ui_hidden,ui_hidden_machines,ui_hidden_facilities,machine_type
0,Machine,,categorical,string,[],,machine__source,NaN,NaN,NaN,NaN,NaN,NaN
1,Cycle Start Time,,datetime,datetime,[],,starttime,NaN,NaN,NaN,NaN,NaN,NaN
2,Cycle End Time,,datetime,datetime,[],,endtime,NaN,NaN,NaN,NaN,NaN,NaN
3,Production Day,,date,datetime,[],,shift_date,NaN,NaN,NaN,NaN,NaN,NaN
4,Cycle Time (Net),ms,continuous,int,[],,total,"{'duration': True, 'formatString': 'seconds'}",NaN,NaN,NaN,NaN,NaN


In [60]:
# example: look at all the various data types for this model
print(schema['type'].unique())

['string' 'datetime' 'int' 'float']


In [61]:
# example: extract list of tags with numeric data types
schema_numeric = schema[schema['type'].apply(lambda x: x in ['int', 'float'])]
cols = schema['display'].to_list()
cols[:10]

['Machine',
 'Cycle Start Time',
 'Cycle End Time',
 'Production Day',
 'Cycle Time (Net)',
 'Cycle Time (Gross)',
 'Shift',
 'Output',
 'Alarms',
 'BLOCKED']

In [66]:
# alternate method
schema_str = cli.get_machine_schema(machines[0], types=['continuous'])
schema_str.head()

,display,unit,sight_type,type,stream_types,raw_data_field,formatting,name,annotations,ui_hidden,ui_hidden_machines,ui_hidden_facilities,machine_type
0,Cycle Time (Net),ms,continuous,int,[],,"{'duration': True, 'formatString': 'seconds'}",total,NaN,NaN,NaN,NaN,NaN
1,Cycle Time (Gross),ms,continuous,int,[],,"{'duration': True, 'formatString': 'seconds'}",record_time,NaN,NaN,NaN,NaN,NaN
2,Alarms,,continuous,float,[cycle],Alarms,{'is_convertible': False},stats__Alarms__val,{},False,[],[],"{'id': 'd9646da3110a08b39b94e34b', 'name': 'La..."
3,BLOCKED,,continuous,float,[cycle],BLOCKED,{'is_convertible': False},stats__BLOCKED__val,{},False,[],[],"{'id': 'd9646da3110a08b39b94e34b', 'name': 'La..."
4,Conveyor Input - Total Time,millisecond,continuous,float,[],ConveyorInput,{'is_convertible': True},stats__ConveyorInput__val,{},False,[],[],"{'id': 'd9646da3110a08b39b94e34b', 'name': 'La..."


## Machine-Type-Level Info

### Get Machine Type Schema (```get_fields_of_machine_type```)
This function is very similar to the above get_machine_schema, except it gets fields that are part of a machine type definition. This function has the same optional parameters as above:
- ```types```: list of strings specifying a subset of column data types that you want to see.
    - ```cli.get_machine_schema(machine_name, types=['continuous'])```
- ```show_hidden```: (default = False) set to True to see the few additional fields that are hidden by default both here and in MA.

In [71]:
type_dict = cli.get_fields_of_machine_type(types[0])
type_schema = pd.DataFrame(type_dict)
type_schema.head()

,display_name,unit,type,data_type,stream_types,raw_data_field,name,formatting,annotations,ui_hidden,ui_hidden_machines,ui_hidden_facilities,machine_type
0,Machine,,categorical,string,[],,machine__source,NaN,NaN,NaN,NaN,NaN,NaN
1,Cycle Start Time,,datetime,datetime,[],,starttime,NaN,NaN,NaN,NaN,NaN,NaN
2,Cycle End Time,,datetime,datetime,[],,endtime,NaN,NaN,NaN,NaN,NaN,NaN
3,Production Day,,date,datetime,[],,shift_date,NaN,NaN,NaN,NaN,NaN,NaN
4,Cycle Time (Net),ms,continuous,int,[],,total,"{'duration': True, 'formatString': 'seconds'}",NaN,NaN,NaN,NaN,NaN
